<a href="https://colab.research.google.com/github/DurjoyAcharya/breast-cancer-detection/blob/main/Breast_Cancer_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.applications import InceptionV3
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import random
import os

In [5]:
Allocation = "/content/drive/MyDrive/SP/UltraSoundDataset"
Groups = ['benign', 'malignant', 'normal']
print(Allocation)
print(Groups)

/content/drive/MyDrive/SP/UltraSoundDataset
['benign', 'malignant', 'normal']


In [6]:
IMAGE_SIZE = (128, 128)

In [11]:
data = []
CATEGORIES = sorted(list(os.listdir(Allocation)))
for category in CATEGORIES:
  folder = os.path.join(Allocation, category)
  label = CATEGORIES.index(category)
  for img in os.listdir(folder):
    img_path = os.path.join(folder, img)
    img_arr = cv2.imread(img_path)
    img_arr = cv2.resize(img_arr, IMAGE_SIZE)
    data.append([img_arr, label])

random.shuffle(data)

In [12]:
from sklearn.model_selection import train_test_split
X, y = zip(*data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = [cv2.resize(img, IMAGE_SIZE) for img in X_train]
X_test = [cv2.resize(img, IMAGE_SIZE) for img in X_test]

In [13]:
X_train = np.array(X_train, dtype=np.float32) / 255.0
y_train = np.array(y_train)
X_test = np.array(X_test, dtype=np.float32) / 255.0
y_test = np.array(y_test)

### Incorporating data augmentation strategies

In [14]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)
])

### Generating the model by adopting DenseNet121 and its pre-trained weights.

In [15]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(128, 128, 3), pooling = 'avg')

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

29084464/29084464 [==============================] - 0s 0us/step


In [16]:
# Create a custom classification head
custom_head = tf.keras.Sequential([
#     tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Combine the base model and custom head
dn = Sequential([data_augmentation, base_model, custom_head])

In [17]:
# Define a learning rate schedule function
def lr_schedule(epoch):
    if epoch < 10:
        return 0.001
    elif epoch < 20:
        return 0.0001
    else:
        return 0.00001

# Create a learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

In [18]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Use the callbacks during model training
dn.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
history = dn.fit(X_train, y_train, epochs=60, batch_size=128, validation_split=0.2, callbacks=[early_stopping, lr_scheduler])
dn.summary()

Epoch 1/60
8/8 [==============================] - 101s 11s/step - loss: 0.9422 - accuracy: 0.5659 - val_loss: 0.5235 - val_accuracy: 0.7984 - lr: 0.0010
Epoch 2/60
8/8 [==============================] - 84s 11s/step - loss: 0.6948 - accuracy: 0.6987 - val_loss: 0.4658 - val_accuracy: 0.7945 - lr: 0.0010
Epoch 3/60
8/8 [==============================] - 79s 10s/step - loss: 0.5357 - accuracy: 0.7611 - val_loss: 0.4135 - val_accuracy: 0.8379 - lr: 0.0010
Epoch 4/60
8/8 [==============================] - 90s 12s/step - loss: 0.4541 - accuracy: 0.7948 - val_loss: 0.3879 - val_accuracy: 0.8577 - lr: 0.0010
Epoch 5/60
8/8 [==============================] - 78s 10s/step - loss: 0.4603 - accuracy: 0.8087 - val_loss: 0.4262 - val_accuracy: 0.7945 - lr: 0.0010
Epoch 6/60
8/8 [==============================] - 107s 14s/step - loss: 0.4404 - accuracy: 0.8097 - val_loss: 0.3360 - val_accuracy: 0.8617 - lr: 0.0010
Epoch 7/60
8/8 [==============================] - 82s 10s/step - loss: 0.3843 - accura